# Practice: Head Pose Detector

¡Hola! Vamos de lleno con la Practice de esta semana. 

El objetivo es detectar dónde está mirando una persona fijándose en una foto de su cara.

Para ello, utilizaremos el Dataset que se puede encontrar en http://crowley-coutaz.fr/HeadPoseDataSet/HeadPoseImageDatabase.tar.gz

### 0. Importación de librerías

Como siempre, recuerda importar las librerías que vayas a necesitar. Te hemos dejado las que podrían serte necesarias para este caso.

Es importante que si no sabes para qué puede valer una librería, hagas una búsqueda y entiendas por qué ha sido importada. Es bastante probable que así puedas identificar mejor las que deberás usar en los siguientes pasos.

In [ ]:
#--------------------Librerias--------------------

# os dejamos las básicas, añadid las que convengan

import cv2  # OpenCV 2 for capturing frames from the video
import os  # For managing paths and directories in the project
import shutil  # High level file operations
import numpy as np  # Arrays
import keras  # High level NN API
from PIL import Image, ImageOps # For image processing
from pathlib import Path  # For easily managing paths
from IPython import display  # For displaying images inline with the notebook
from sklearn.model_selection import train_test_split  # For train-test splitting
from tqdm import tqdm
import re
import requests
import pandas as pd
import glob

### 1. Descarga de Dataset: Head Pose Image Database (Gourier, Hall, & Crowley, 2004)

Lo siguiente es descargar el dataset de http://crowley-coutaz.fr/HeadPoseDataSet/HeadPoseImageDatabase.tar.gz y estructurar los datos para dejarlos listos para su uso (descompresión del archivo tar.gz, creación de variables, tratamiento de expresiones regulares ...).

La información relevante a la construcción del dataset se puede encontrar en http://crowley-coutaz.fr/Head%20Pose%20Image%20Database.html

##### 1.1 Descarga y tratamiento como archivo

In [ ]:
url = 'http://crowley-coutaz.fr/HeadPoseDataSet/HeadPoseImageDatabase.tar.gz'
name = 'HeadPoseImageDatabase.tar.gz'

#Insertar a continuación uso de librería requests 




##### 1.2 Descomprime del archivo tar.gz, usando el Linux que hay debajo de Jupyter Notebook, mediante el comando tar

#### Formato del archivo:

Recordad que en el enlace de información se describe cómo está guardada la información, estando en cada archivo de la imagen los ángulos de inclinación y giro (tilt,pan) y las coordenadas de la cara (x,y), altura y anchura (h,w) dentro del archivo.

##### 1.3 Tratamiento de las expresiones regulares de los títulos de las imágenes para conseguir las caracteristicas

Os dejamos esta función para que, dado el path de una imagen, pueda transformarla a un tamaño más trabajable.

In [ ]:
def img_df(image_path, shape):
    image = Image.open(image_path)
    image_resized = image.resize(shape, Image.ANTIALIAS)
    img_array = np.asarray(image_resized)
    return img_array

In [ ]:
# Cargar los datos en el dataframe, extraidos de cada archivo (nombre del archivo y contenidos)

df = pd.DataFrame()









# Finalmente deberíais tener algo como lo siguiente

df.columns = ["X", "Y", "H", "W", "T", "P", "Image"]
df.X = df.X.astype(int)
df.Y = df.Y.astype(int)
df.H = df.H.astype(int)
df.W = df.W.astype(int)
df = df.reset_index().drop("index", axis=1)

##### 1.4 Separado en X (datos) e Y (a predecir) y su normalización

In [ ]:
X = np.asarray(list(df["Image"]/255.))
Y = np.array(df[["X", "Y", "H", "W", "T", "P"]])/100.
X.shape

##### 1.5 Split de conjuntos finales de x_train, y_train, x_test y y_test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

### 2 Importado de red neuronal MobileNet sin incluir la última capa

El objetivo aquí es importar la red neuronal MobileNet (una arquitectura que ha demostrado ser bastante eficiente para este problema) excluyendo la última capa. Con ello, descartaremos la última capa, para posteriormente crearla nosotros y concatenar las dos partes.

##### 2.1 Importar la red de Keras sin la última capa

In [ ]:
# Nosotros como hemos dicho os recomendamos Mobilenet
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.models import Model

n_classes = 6
base_model = MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

##### 2.2 Crear nuestra última capa

##### 2.3 Juntar la red y la capa

In [ ]:
#Añade el código que falta para unir la red y la capa nueva generada.

model=

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

##### 2.4 Compilar (elegir optimizador, funcion de perdida(loss) y métrica de error)

In [ ]:
model.compile(optimizer = '',
              loss ='',
              metrics = [''])

##### 2.5 Entrenar la red que hemos importado y manipulado con el dataset que hemos tratado

In [ ]:
#--------------------Entrenamiento de nuestra red personalizada--------------------
# Un precioso fit() y a esperar. Unas 10 épocas deberian dar un resultado decente
model.fit(X_train, Y_train, validation_data=[X_test, Y_test], epochs=10, verbose=1)

##### 2.6 Visualizar un diagrama de correlación entre los valores predichos y los valores que debieran ser (usando por ejemplo el RMSE o R2 dado por sklearn)

In [ ]:
from sklearn.metrics import r2_score

